In [1]:
from google.colab import drive
# Step 1: Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np

# Define dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/train"  # Adjust path
depths_dir = os.path.join(dataset_root, "depths")  # Folder containing MiDaS depth images
labels_dir = os.path.join(dataset_root, "labels")  # Folder containing YOLO labels

# New folder for modified labels (depth_labels)
depth_labels_dir = os.path.join(dataset_root, "depth_labels")
os.makedirs(depth_labels_dir, exist_ok=True)

# Get list of depth images for quick lookup
depth_image_files = set(os.listdir(depths_dir))

# Process each label file
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):
        label_path = os.path.join(labels_dir, label_file)
        depth_filename = label_file.replace(".txt", ".jpg")  # MiDaS depth maps are often in PNG format

        # Check if corresponding depth image exists
        if depth_filename not in depth_image_files:
            print(f"⚠️ No matching depth image for {label_file}, skipping...")
            continue

        depth_path = os.path.join(depths_dir, depth_filename)

        # Load depth image (grayscale)
        depth_map = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)  # Load actual depth values

        if depth_map is None or depth_map.size == 0:
            print(f"⚠️ Error loading depth image for {label_file}, skipping...")
            continue

        # Invert the depth map to ensure closer objects have smaller values
        depth_map = np.max(depth_map) - depth_map

        updated_lines = []
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Get image dimensions
        img_height, img_width = depth_map.shape

        # Process each bounding box in the label file
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # Skip invalid lines

            class_id, x_center, y_center, width, height = map(float, parts[:5])

            # Convert normalized YOLO coordinates to pixel coordinates
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)

            # Ensure bounding box coordinates are within image bounds
            x_min, y_min = max(0, x_min), max(0, y_min)
            x_max, y_max = min(img_width - 1, x_max), min(img_height - 1, y_max)

            # Extract depth values within the bounding box
            depth_region = depth_map[y_min:y_max, x_min:x_max]

            # Ensure depth region is valid
            if depth_region.size > 0:
                min_depth = np.min(depth_region)  # Use minimum depth (closest point)
                avg_depth = np.mean(depth_region)  # Use average depth for verification
            else:
                min_depth = 0.0  # Default if no valid depth
                avg_depth = 0.0

            # Normalize min_depth so that closer objects get **smaller** values
            final_depth = min_depth / np.max(depth_map)

            # Append depth value to label file
            updated_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {final_depth:.6f}\n"
            updated_lines.append(updated_line)

        # Save updated labels in the new depth_labels folder (KEEPING THE SAME FILE NAME)
        output_label_path = os.path.join(depth_labels_dir, label_file)
        with open(output_label_path, "w") as f:
            f.writelines(updated_lines)

        print(f"✅ Depth-annotated label saved: {output_label_path}")

print("🎯 Depth values successfully added and saved in 'depth_labels/' folder!")

✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/train/depth_labels/positive_w1-26_frame_270_jpg.rf.0ce20d77f49c11b1760a218fd81b03dd.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/train/depth_labels/positive_w1-26_frame_0_jpg.rf.ed2d2bc474aa59625e529ce0637f13fb.txt
⚠️ No matching depth image for positive_w1-26_frame_180_jpg.rf.0bde87726d602c8fff928ad5b51fa592.txt, skipping...
⚠️ No matching depth image for positive_w1-26_frame_90_jpg.rf.3f65b625160041b8682e435d6013d6ad.txt, skipping...
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/train/depth_labels/positive_w1_11_frame_270_jpg.rf.e56033381da1afbf4358434037f65dca.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/train/depth_labels/positive_w1-34_frame_90_jpg.rf.7b34448a35c41ede46f4af4d18d4a00c.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/train/depth_labels/positive_w1-34_frame_270_jpg.rf.d9a2df991b

In [3]:
import os
import cv2
import numpy as np

# Define dataset paths
dataset_root = "/content/drive/MyDrive/TharuWorks/Dataset/val"  # Adjust path
depths_dir = os.path.join(dataset_root, "depths")  # Folder containing MiDaS depth images
labels_dir = os.path.join(dataset_root, "labels")  # Folder containing YOLO labels

# New folder for modified labels (depth_labels)
depth_labels_dir = os.path.join(dataset_root, "depth_labels")
os.makedirs(depth_labels_dir, exist_ok=True)

# Get list of depth images for quick lookup
depth_image_files = set(os.listdir(depths_dir))

# Process each label file
for label_file in os.listdir(labels_dir):
    if label_file.endswith(".txt"):
        label_path = os.path.join(labels_dir, label_file)
        depth_filename = label_file.replace(".txt", ".jpg")  # MiDaS depth maps are often in PNG format

        # Check if corresponding depth image exists
        if depth_filename not in depth_image_files:
            print(f"⚠️ No matching depth image for {label_file}, skipping...")
            continue

        depth_path = os.path.join(depths_dir, depth_filename)

        # Load depth image (grayscale)
        depth_map = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH)  # Load actual depth values

        if depth_map is None or depth_map.size == 0:
            print(f"⚠️ Error loading depth image for {label_file}, skipping...")
            continue

        # Invert the depth map to ensure closer objects have smaller values
        depth_map = np.max(depth_map) - depth_map

        updated_lines = []
        with open(label_path, "r") as f:
            lines = f.readlines()

        # Get image dimensions
        img_height, img_width = depth_map.shape

        # Process each bounding box in the label file
        for line in lines:
            parts = line.strip().split()
            if len(parts) < 5:
                continue  # Skip invalid lines

            class_id, x_center, y_center, width, height = map(float, parts[:5])

            # Convert normalized YOLO coordinates to pixel coordinates
            x_min = int((x_center - width / 2) * img_width)
            y_min = int((y_center - height / 2) * img_height)
            x_max = int((x_center + width / 2) * img_width)
            y_max = int((y_center + height / 2) * img_height)

            # Ensure bounding box coordinates are within image bounds
            x_min, y_min = max(0, x_min), max(0, y_min)
            x_max, y_max = min(img_width - 1, x_max), min(img_height - 1, y_max)

            # Extract depth values within the bounding box
            depth_region = depth_map[y_min:y_max, x_min:x_max]

            # Ensure depth region is valid
            if depth_region.size > 0:
                min_depth = np.min(depth_region)  # Use minimum depth (closest point)
                avg_depth = np.mean(depth_region)  # Use average depth for verification
            else:
                min_depth = 0.0  # Default if no valid depth
                avg_depth = 0.0

            # Normalize min_depth so that closer objects get **smaller** values
            final_depth = min_depth / np.max(depth_map)

            # Append depth value to label file
            updated_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} {final_depth:.6f}\n"
            updated_lines.append(updated_line)

        # Save updated labels in the new depth_labels folder (KEEPING THE SAME FILE NAME)
        output_label_path = os.path.join(depth_labels_dir, label_file)
        with open(output_label_path, "w") as f:
            f.writelines(updated_lines)

        print(f"✅ Depth-annotated label saved: {output_label_path}")

print("🎯 Depth values successfully added and saved in 'depth_labels/' folder!")

✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w10_2_frame_90_jpg.rf.d1b2aec1cecc98ac228c201d7fe568de.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w2_53_frame_270_jpg.rf.63dc5545897fb1134110e0dfc0beca64.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w10_30_frame_90_jpg.rf.da25d9eebba985ddea66810b021e74ec.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w10_41_frame_0_jpg.rf.643af631b91c611776bb4143b2f9c93d.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w3_35_frame_90_jpg.rf.f8f8a6ad3ef1ed54a0fd5223b4e90a81.txt
✅ Depth-annotated label saved: /content/drive/MyDrive/TharuWorks/Dataset/val/depth_labels/negative_w3_35_frame_180_jpg.rf.2072e7852c4a06c300e79edc15b5815c.txt
✅ Depth-annotated label saved: /content/drive/MyD